# Semester 1 Project Submission



## Elliot

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## Ian

### Items to be Resolved:
- Check for nulls in the sql database tables being used
    - See following cell for findings.
- Resolve found nulls 
    - Rows where null can be excluded. This is due to size and number of entries in dataset tables.
- Experiment with joins to get tables wanted
    - Joined movie_basics with movie_ratings. Tried further joins but they get very large, very quickly (even using inner joins).
- Export joined tables to dataframe
    - Complete.
- Determine how to best use data for analysis
    - Complete.
    
### Hypothesis Testing Checklist
- Test runtime to domestic revenue and worldwide revenue - Complete.
    - Runtime to revenue would assist with recommending best length of film to make for optimal revenue outlook
- Break down runtime into buckets of time periods to calculate for correlation - Complete.
    - Will isolate periods with best correlation to complement observations from runtime and revenue correlation tests.
____________

### Data Observations and Exploration

- persons % bunch of null values in birth_year, death_year, and in primary_profession
- writers % NONE ARE NULL
- directors % NONE ARE NULL
- known_for % NONE ARE NULL
- principals % NO NULLS IN CATEGORY
- movie_basics % missing data in genres, runtime_minutes
- movie_ratings % NONE ARE NULL
- movie_akas % NO TITLES ARE NULL

In [ ]:
# creating connection to database
conn = sql.connect("databases/im.db")

In [ ]:
# movie basics query stage 1
q = """
SELECT
    *
FROM
    movie_basics
/*where statement below reduces dataset from 146144 rows to 112233*/
WHERE
    runtime_minutes IS NOT NULL AND genres IS NOT NULL
;"""
pd.read_sql(q, conn)

In [ ]:
# test to see how many values of each genre there are
q = """
SELECT
    *
FROM
    movie_basics
/*where statement below reduces dataset from 146144 rows to 112233*/
WHERE
    runtime_minutes IS NOT NULL AND genres IS NOT NULL
;"""
test2_df = pd.read_sql(q, conn)
test2_df.value_counts('genres')

In [ ]:
# movie ratings query stage 1
q = """
SELECT
    *
FROM
    movie_ratings
/*where statement below reduces dataset from 73856 rows to 73856 NO NULLS*/
WHERE
    averagerating IS NOT NULL AND numvotes IS NOT NULL
;"""
pd.read_sql(q, conn)

In [ ]:
# movie_akas query stage 1
q = """
SELECT
    *
FROM
    movie_akas
/*where statement below reduces dataset from 331703 rows to 331703 NO NULLS*/
WHERE
    title IS NOT NULL
;"""
pd.read_sql(q, conn)

In [ ]:
# persons table query stage 1
q = """
SELECT
    *
FROM
    persons
WHERE
    primary_profession IS NULL
;"""
pd.read_sql(q, conn)

In [ ]:
# principals exploration query
q = """
SELECT
    *
FROM
    principals
;"""
pd.read_sql(q, conn)

In [ ]:
# principals category query
q = """
SELECT
    movie_id, person_id, category
FROM
    principals
;"""
test1_df = pd.read_sql(q, conn)
test1_df.value_counts('category')

In [ ]:
# directors exploration query
q = """
SELECT
    *
FROM
    directors
;"""
pd.read_sql(q, conn)

In [ ]:
# known_for exploration query
q = """
SELECT
    *
FROM
    known_for
;"""
pd.read_sql(q, conn)

______________________________

### Consolidation Queries

In [ ]:
# combining movie basics and movie ratings
q = """
SELECT
    mb.primary_title
    ,mb.runtime_minutes
    ,mb.genres
    ,mr.averagerating AS AverageRating
    ,mr.numvotes AS Num_Votes
FROM
    movie_basics AS mb
    JOIN movie_ratings AS mr
        USING(movie_id)
WHERE
    CAST(Num_Votes AS int) > 30
    AND mb.runtime_minutes IS NOT NULL
ORDER BY
    mr.averagerating DESC
;"""
pd.read_sql(q, conn)
# added in a filter that would filter out rows that have less than 30 votes

In [ ]:
# combining movie basics and movie ratings
q = """
SELECT
    mb.primary_title, mb.runtime_minutes, mb.genres, mr.averagerating
FROM
    movie_basics AS mb
    JOIN movie_ratings AS mr
        USING(movie_id)
WHERE
    CAST(averagerating AS float) == 10
    AND runtime_minutes IS NOT NULL
ORDER BY
    mr.averagerating DESC
;"""
# checking to see titles with a rating of 10.0
# update to above comment: restricting num_votes by 30 (like above) will
test3_df = pd.read_sql(q, conn)
test3_df

In [ ]:
# persons to principals to movie table NO FILTERS
q = """
SELECT 
    pe.person_id
    ,pe.primary_name
    ,AVG(mr.averagerating) AS AverageRating
    ,COUNT(mb.primary_title) AS Num_of_Appearances
FROM
    persons AS pe
    JOIN principals AS pr
        USING(person_id)
    JOIN movie_basics AS mb
        USING(movie_id)
    JOIN movie_ratings as mr
        USING(movie_id)
GROUP BY
    pe.primary_name
ORDER BY
    Num_of_Appearances DESC
LIMIT 10
;"""
pd.read_sql(q, conn)

In [ ]:
# finding the average ratings per film and num of appearances for writers
q = """
SELECT 
    pe.person_id
    ,pr.category
    ,pe.primary_name
    ,AVG(mr.averagerating) AS AverageRating
    ,COUNT(mb.primary_title) AS Num_of_Appearances
FROM
    persons AS pe
    JOIN principals AS pr
        USING(person_id)
    JOIN movie_basics AS mb
        USING(movie_id)
    JOIN movie_ratings as mr
        USING(movie_id)
WHERE
    pr.category = 'writer'
    
GROUP BY
    pe.primary_name
ORDER BY
    Num_of_Appearances DESC
;"""
pd.read_sql(q, conn)

In [ ]:
# finding the average ratings per film and num of appearances for directors
q = """
SELECT 
    pe.person_id
    ,pr.category
    ,pe.primary_name
    ,AVG(mr.averagerating) AS AverageRating
    ,COUNT(mb.primary_title) AS Num_of_Appearances
FROM
    persons AS pe
    JOIN principals AS pr
        USING(person_id)
    JOIN movie_basics AS mb
        USING(movie_id)
    JOIN movie_ratings as mr
        USING(movie_id)
WHERE
    pr.category = 'director'
    
GROUP BY
    pe.primary_name
HAVING
    CAST(Num_of_Appearances AS int) > 1
ORDER BY
    Num_of_Appearances DESC
;"""
pd.read_sql(q, conn)

In [ ]:
# finding the average ratings per film and num of appearances for lead actor/actress
# lead actor/actress is found by filtering on principals' column called ordering for the 1st actor
q = """
SELECT 
    pe.person_id
    ,pr.category
    ,pe.primary_name
    ,AVG(mr.averagerating) AS AverageRating
    ,COUNT(mb.primary_title) AS Num_of_Appearances
FROM
    persons AS pe
    JOIN principals AS pr
        USING(person_id)
    JOIN movie_basics AS mb
        USING(movie_id)
    JOIN movie_ratings as mr
        USING(movie_id)
WHERE
    pr.category IN ('actor', 'actress')
    AND CAST(AverageRating AS float) >= 6
    AND CAST(pr.ordering AS int) == 1
GROUP BY
    pe.primary_name
HAVING
    CAST(Num_of_Appearances AS int) > 2
ORDER BY
    Num_of_Appearances DESC
;"""
# filtering for only actor/actresses who had an average rating of 6 or greater
pd.read_sql(q, conn)

In [ ]:
# finding the average ratings per film and num of appearances for actor/actress
q = """
SELECT 
    pe.person_id
    ,pr.category
    ,pe.primary_name
    ,AVG(mr.averagerating) AS AverageRating
    ,COUNT(mb.primary_title) AS Num_of_Appearances
FROM
    persons AS pe
    JOIN principals AS pr
        USING(person_id)
    JOIN movie_basics AS mb
        USING(movie_id)
    JOIN movie_ratings as mr
        USING(movie_id)
WHERE
    pr.category IN ('actor', 'actress')
    AND CAST(AverageRating AS float) >= 6
GROUP BY
    pe.primary_name
HAVING
    CAST(Num_of_Appearances AS int) > 2
ORDER BY
    Num_of_Appearances DESC
;"""
# filtering for only actor/actresses who had an average rating of 6 or greater
pd.read_sql(q, conn)

In [ ]:
# finding the average ratings per film and num of appearances for producer
# hard to quantify where to limit here regarding appearances -- producers often only produce once, apparently
q = """
SELECT 
    pe.person_id
    ,pr.category
    ,pe.primary_name
    ,AVG(mr.averagerating) AS AverageRating
    ,COUNT(mb.primary_title) AS Num_of_Appearances
FROM
    persons AS pe
    JOIN principals AS pr
        USING(person_id)
    JOIN movie_basics AS mb
        USING(movie_id)
    JOIN movie_ratings as mr
        USING(movie_id)
WHERE
    pr.category = 'producer'
    AND CAST(AverageRating AS float) >= 6
GROUP BY
    pe.primary_name
HAVING
    CAST(Num_of_Appearances AS int) > 1
ORDER BY
    AverageRating DESC
;"""
pd.read_sql(q, conn)

## How to Check for Statistically Significant Correlation

1. State your null and alternative hypotheses, along with your alpha.
2. Retrieve the sample being tested
3. Calculate the test statistic, r
4. Calculate degrees of freedom and find r-critical using table or calculator (source I used since we have large datasets with degrees of freedom that are hard to find on a table and I could not locate a stastical package to calculate the r-critical in-notebook): https://www.calculators.tech/t-value-calculator)
5. If the absolute value of r is greater than r-critical, the correlation is statistically significant and we reject the null hypothesis. If the absolute value of r is less than r-critical, the correlation is not statistically signifcant and we fail to reject the null hypothesis.

### Testing Runtime and Average Rating

In [ ]:
# added WHERE clause to remove movies from dataframe if longer than 3.5 hours - took away 65 values from dataset
# and reduced runtime mean, std dev.!
q = """
SELECT
    mb.movie_id
    ,mb.primary_title AS title
    ,mb.runtime_minutes AS Runtime
    ,mb.genres
    ,mr.averagerating AS AverageRating
    ,mr.numvotes AS Num_Votes
FROM
    movie_basics AS mb
    JOIN movie_ratings AS mr
        USING(movie_id)
WHERE
    CAST(Num_Votes AS int) > 30
    AND CAST(mb.runtime_minutes as float) <= 210
    AND mb.runtime_minutes IS NOT NULL
    AND mb.genres IS NOT NULL
ORDER BY
    mb.runtime_minutes DESC
;"""
df_allrt = pd.read_sql(q, conn)
df_allrt

In [ ]:
df_allrt.duplicated().value_counts() 

### Ho:
##### - There is no correlation between runtime and average rating. 

### Ha:
##### - There is a statistically significant correlation between runtime and average rating at the 95% level. 

In [ ]:
df = df_allrt['Runtime'].count() - 2 # getting degrees of freedom to determine r-critical
df

In [ ]:
# r-critical value to test if found r is statistically signifcant
critical_r = 0.0081
# Source: https://www.calculators.tech/t-value-calculator
# (Unable to calculate by hand as I have been unable to find a direct formula to find this - must use either r-critical table or a r-crit calculator like the above source)

In [ ]:
# calculating correlation

x = df_allrt['Runtime']
y = df_allrt['AverageRating']

r = x.corr(y) # result is the pearson correlation coefficient -- seems we do have a positive correlation!
r

print(f'Pearson Correlation: {r} | r-critical: {critical_r} | Is r greater? {abs(r) >= critical_r}')
if abs(r) >= critical_r:
    print("Reject the null")
elif abs(r) < critical_r:
    print("Fail to reject null")

In [ ]:
sns.set_style("whitegrid")
sns.set_context("talk")
sns.lmplot(x="Runtime", y="AverageRating", data=df_allrt, line_kws={'color': 'red'}, height=8, aspect=1.5);

#### Conclusion:
- For 95% of cases, there is a weak positive correlation of 0.1014 between runtime and rating.
- Now, we will drill in further to observe individual groupings

### Operating Notion with Runtime Samples:
- "The Academy of Motion Picture Arts and Sciences defines a feature as a film that runs for more than 40 minutes (with short films being 40 or fewer minutes). Still, the Screen Actors Guild asserts that a feature's running time is 60 minutes, so there's not exactly a widespread agreement."

Source: https://www.arcstudiopro.com/blog/what-is-a-feature-film#:~:text=The%20Academy%20of%20Motion%20Picture,not%20exactly%20a%20widespread%20agreement

In [ ]:
# standardizing population for secondary visualizations to follow
mu = df_allrt['AverageRating'].mean()
std = df_allrt['AverageRating'].std()

pop_standardized_rating = [(x-mu)/std for x in df_allrt['AverageRating']]
pop_z_mean = np.mean(pop_standardized_rating)
pop_z_mean

### Less than 40 Minutes Testing

### Z-Test

### Ho:
##### - There is no difference between the average rating of a movie that has a runtime of less than 40 minutes and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the average rating of a movie that has a runtime of less than 40 minutes and the population. 

In [ ]:
# runtime sample 1 - short films (between 0 and 40 minutes)
q = """
SELECT
    mb.movie_id
    ,mb.primary_title AS title
    ,mb.runtime_minutes AS Runtime
    ,mb.genres
    ,mr.averagerating AS AverageRating
    ,mr.numvotes AS Num_Votes
FROM
    movie_basics AS mb
    JOIN movie_ratings AS mr
        USING(movie_id)
WHERE
    CAST(Num_Votes AS int) > 30
    AND CAST(mb.runtime_minutes as float) <= 40
    AND mb.runtime_minutes IS NOT NULL
    AND mb.genres IS NOT NULL
    AND mb.primary_title != 'The Death of a Security Guard'
ORDER BY
    mb.runtime_minutes DESC
;"""
df_up40 = pd.read_sql(q, conn)
df_up40

In [ ]:
# z-test - less than 40 minutes ratings run

# two-tailed z-test

a = 0.025

x_bar = df_up40['AverageRating'].mean()
n = df_up40['AverageRating'].count()
std = df_allrt['AverageRating'].std()
mu = df_allrt['AverageRating'].mean()

z = (x_bar - mu)/(std/np.sqrt(n))
p = 1 - stats.norm.cdf(np.absolute(z))

# displaying results
print(f'Sample Mean: {x_bar} | Population Mean: {mu} | Sample Size: {n}')
print(f'p-value: {p} | z-score: {z} | Is p less than alpha? {p < a}')
if p < a:
    print("Reject the null")
elif p >= a:
    print("Fail to reject null")

In [ ]:
# normalizing distribution with for loop
samp_standardized_rating = [(x-mu)/std for x in df_up40['AverageRating']]
z_mean = np.mean(samp_standardized_rating)
print(f' Standardized Sample Mean: {z_mean}')

# visualizing above distribution as histogram
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(samp_standardized_rating, bins=11, density=True, color='r', alpha=0.9)
ax.set_xlabel('Z Scores')
ax.set_title('Less than 40 Minutes - Ratings')

# applying normalized curve over histogram
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

# marking standardized_mean
ax.vlines(x=z_mean, ymin=0, ymax=0.7, color='b', label='Mean: 0.7702')
ax.legend()

In [ ]:
# plotting tested runtime sample against population
fig, ax = plt.subplots(figsize=(10,8))

plt.hist(pop_standardized_rating, bins=45, density=True, color='b', alpha = 0.5)
sns.kdeplot(pop_standardized_rating, ax=ax, color='y')

plt.hist(samp_standardized_rating, bins=11, density=True, color='r', alpha = 0.5)
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

plt.vlines(x=z_mean, ymin=0, ymax=0.7, color='m', label='x<=40 Mean: 0.7702')
plt.vlines(x=pop_z_mean, ymin=0, ymax=0.7, label='Population Mean: 0.0')
                                              
plt.xlabel('Z Scores')
plt.title('Less than 40 Minutes to All Films - Ratings')
plt.legend()

### Correlation

### Ho:
##### - There is no correlation between runtime and average rating for movies that are less than 40 minutes. 

### Ha:
##### - There is a statistically significant correlation between runtime and average rating at the 95% level for movies that are less than 40 minutes. 

In [ ]:
# r-critical value to test if found r is statistically signifcant
critical_r = 0.2510
# Source: https://www.calculators.tech/t-value-calculator
# (Unable to calculate by hand as I have been unable to find a direct formula to find this - must use either r-critical table or a r-crit calculator like the above source)
df = df_up40['Runtime'].count() - 2 # getting degrees of freedom to determine r-critical
df

In [ ]:
# calculating correlation

x = df_up40['Runtime']
y = df_up40['AverageRating']

r = x.corr(y) # result is the pearson correlation coefficient
r

print(f'Pearson Correlation: {r} | r-critical: {critical_r} | Is r greater? {abs(r) >= critical_r}')
if abs(r) >= critical_r:
    print("Reject the null")
elif abs(r) < critical_r:
    print("Fail to reject null")

In [ ]:
sns.lmplot(x="Runtime", y="AverageRating", data=df_up40, line_kws={'color': 'red'}, height=8, aspect=1.5);

### 40 to 60 Minutes Testing

### Z-Test

### Ho:
##### - There is no difference between the average rating of a movie that has a runtime of between 40 minutes and 60 minutes and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the average rating of a movie that has a runtime of between 40 minutes and 60 minutes and other movies in the population.

In [ ]:
# runtime sample 2 - long-form short films (40.0001 minutes to 60 minutes)
q = """
SELECT
    mb.movie_id
    ,mb.primary_title AS title
    ,mb.runtime_minutes AS Runtime
    ,mb.genres
    ,mr.averagerating AS AverageRating
    ,mr.numvotes AS Num_Votes
FROM
    movie_basics AS mb
    JOIN movie_ratings AS mr
        USING(movie_id)
WHERE
    CAST(Num_Votes AS int) > 30
    AND CAST(mb.runtime_minutes as float) BETWEEN 40.0001 AND 60
    AND mb.runtime_minutes IS NOT NULL
    AND mb.genres IS NOT NULL
ORDER BY
    mb.runtime_minutes DESC
;"""
df_4060 = pd.read_sql(q, conn)
df_4060

In [ ]:
# z-test - 40 to 60 minutes ratings run

a = 0.025

# two-tailed z-test

x_bar = df_4060['AverageRating'].mean()
n = df_4060['AverageRating'].count()
std = df_allrt['AverageRating'].std()
mu = df_allrt['AverageRating'].mean()

z = (x_bar - mu)/(std/np.sqrt(n))
p = 1 - stats.norm.cdf(np.absolute(z))

# displaying results
print(f'Sample Mean: {x_bar} | Population Mean: {mu} | Sample Size: {n}')
print(f'p-value: {p} | z-score: {z} | Is p less than alpha? {p < a}')
if p < a:
    print("Reject the null")
elif p >= a:
    print("Fail to reject null")

In [ ]:
# normalizing distribution with for loop
samp_standardized_rating = [(x-mu)/std for x in df_4060['AverageRating']]
z_mean = np.mean(samp_standardized_rating)
print(f' Standardized Sample Mean: {z_mean}')

In [ ]:
# visualizing above distribution as histogram
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(samp_standardized_rating, bins=15, density=True, color='r')
ax.set_xlabel('Z Scores')
ax.set_title('40 to 60 Minutes - Ratings')

# applying normalized curve over histogram
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

# marking standardized_mean
ax.vlines(x=z_mean, ymin=0, ymax=0.7, color='b', label='Mean: 0.6132')
ax.legend()

In [ ]:
# plotting tested runtime sample against population
fig, ax = plt.subplots(figsize=(10,8))

plt.hist(pop_standardized_rating, bins=45, density=True, color='b', alpha = 0.5)
sns.kdeplot(pop_standardized_rating, ax=ax, color='y')

plt.hist(samp_standardized_rating, bins=15, density=True, color='r', alpha = 0.5)
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

plt.vlines(x=z_mean, ymin=0, ymax=0.7, color='m', label='40 < x <= 60 Mean: 0.6132')
plt.vlines(x=pop_z_mean, ymin=0, ymax=0.7, label='Population Mean: 0.0')
                                              
plt.xlabel('Z Scores')
plt.title('40 to 60 Minutes to All Films - Ratings')
plt.legend()

### Correlation

### Ho:
##### - There is no correlation between runtime and average rating for movies that are between 40 and 60 minutes. 

### Ha:
##### - There is a statistically significant correlation between runtime and average rating at the 95% level for movies that are between 40 and 60 minutes. 

In [ ]:
# r-critical value to test if found r is statistically signifcant
critical_r = 0.0529
# Source: https://www.calculators.tech/t-value-calculator
# (Unable to calculate by hand as I have been unable to find a direct formula to find this - must use either r-critical table or a r-crit calculator like the above source)
df = df_4060['Runtime'].count() - 2 # getting degrees of freedom to determine r-critical
df

In [ ]:
# calculating correlation

x = df_4060['Runtime']
y = df_4060['AverageRating']

r = x.corr(y) # result is the pearson correlation coefficient
r

print(f'Pearson Correlation: {r} | r-critical: {critical_r} | Is r greater? {abs(r) >= critical_r}')
if abs(r) >= critical_r:
    print("Reject the null")
elif abs(r) < critical_r:
    print("Fail to reject null")

In [ ]:
sns.lmplot(x="Runtime", y="AverageRating", data=df_4060, line_kws={'color': 'red'}, height=8, aspect=1.5);

### 60 to 90 Minutes Testing

### Z-Test

### Ho:
##### - There is no difference between the average rating of a movie that has a runtime of between 60 minutes and 90 minutes and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the average rating of a movie that has a runtime of between 60 minutes and 90 minutes and other movies in the population.

In [ ]:
# runtime sample 3 - short-mid length films (60.0001 minutes to 90 minutes)
q = """
SELECT
    mb.movie_id
    ,mb.primary_title AS title
    ,mb.runtime_minutes AS Runtime
    ,mb.genres
    ,mr.averagerating AS AverageRating
    ,mr.numvotes AS Num_Votes
FROM
    movie_basics AS mb
    JOIN movie_ratings AS mr
        USING(movie_id)
WHERE
    CAST(Num_Votes AS int) > 30
    AND CAST(mb.runtime_minutes as float) BETWEEN 60.0001 AND 90
    AND mb.runtime_minutes IS NOT NULL
    AND mb.genres IS NOT NULL
ORDER BY
    mb.runtime_minutes DESC
;"""
df_6090 = pd.read_sql(q, conn)
df_6090

In [ ]:
# z-test - 60 to 90 minutes ratings run

a = 0.025

# two-tailed z-test

x_bar = df_6090['AverageRating'].mean()
n = df_6090['AverageRating'].count()
std = df_allrt['AverageRating'].std()
mu = df_allrt['AverageRating'].mean()

z = (x_bar - mu)/(std/np.sqrt(n))
p = 1 - stats.norm.cdf(np.absolute(z))

# displaying results
print(f'Sample Mean: {x_bar} | Population Mean: {mu} | Sample Size: {n}')
print(f'p-value: {p} | z-score: {z} | Is p less than alpha? {p < a}')
if p < a:
    print("Reject the null")
elif p >= a:
    print("Fail to reject null")

In [ ]:
# normalizing distribution with for loop
samp_standardized_rating = [(x-mu)/std for x in df_6090['AverageRating']]
z_mean = np.mean(samp_standardized_rating)
print(f' Standardized Sample Mean: {z_mean}')

In [ ]:
# visualizing above distribution as histogram
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(samp_standardized_rating, bins=22, density=True, color='r')
ax.set_xlabel('Z Scores')
ax.set_title('60 to 90 Minutes - Ratings')

# applying normalized curve over histogram
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

# marking standardized_mean
ax.vlines(x=z_mean, ymin=0, ymax=0.7, color='b', label='Mean: -0.1648')
ax.legend()

In [ ]:
# plotting tested runtime sample against population
fig, ax = plt.subplots(figsize=(10,8))

plt.hist(pop_standardized_rating, bins=45, density=True, color='b', alpha = 0.5)
sns.kdeplot(pop_standardized_rating, ax=ax, color='y')

plt.hist(samp_standardized_rating, bins=22, density=True, color='r', alpha = 0.5)
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

plt.vlines(x=z_mean, ymin=0, ymax=0.7, color='m', label='60 < x <= 90 Mean: -0.1648')
plt.vlines(x=pop_z_mean, ymin=0, ymax=0.7, label='Population Mean: 0.0')
                                              
plt.xlabel('Z Scores')
plt.title('60 to 90 Minutes to All Films - Ratings')
plt.legend()

### Correlation

### Ho:
##### - There is no correlation between runtime and average rating for movies that are between 60 and 90 minutes. 

### Ha:
##### - There is a statistically significant correlation between runtime and average rating at the 95% level for movies that are between 60 and 90 minutes. 

In [ ]:
# r-critical value to test if found r is statistically signifcant
critical_r = 0.0132
# Source: https://www.calculators.tech/t-value-calculator
# (Unable to calculate by hand as I have been unable to find a direct formula to find this - must use either r-critical table or a r-crit calculator like the above source)
df = df_6090['Runtime'].count() - 2 # getting degrees of freedom to determine r-critical
df

In [ ]:
# calculating correlation

x = df_6090['Runtime']
y = df_6090['AverageRating']

r = x.corr(y) # result is the pearson correlation coefficient
r

print(f'Pearson Correlation: {r} | r-critical: {critical_r} | Is r greater? {abs(r) >= critical_r}')
if abs(r) >= critical_r:
    print("Reject the null")
elif abs(r) < critical_r:
    print("Fail to reject null")

In [ ]:
sns.lmplot(x="Runtime", y="AverageRating", data=df_6090, line_kws={'color': 'red'}, height=8, aspect=1.5);

### 90 to 120 Minutes Testing

### Z-Test

### Ho
##### - There is no difference between the average rating of a movie that has a runtime of between 90 minutes and 120 minutes and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the average rating of a movie that has a runtime of between 90 minutes and 120 minutes and other movies in the population.

In [ ]:
# runtime sample 4 - relative average length films (90.0001 minutes to 120 minutes)
q = """
SELECT
    mb.movie_id
    ,mb.primary_title AS title
    ,mb.runtime_minutes AS Runtime
    ,mb.genres
    ,mr.averagerating AS AverageRating
    ,mr.numvotes AS Num_Votes
FROM
    movie_basics AS mb
    JOIN movie_ratings AS mr
        USING(movie_id)
WHERE
    CAST(Num_Votes AS int) > 30
    AND CAST(mb.runtime_minutes as float) BETWEEN 90.0001 AND 120
    AND mb.runtime_minutes IS NOT NULL
    AND mb.genres IS NOT NULL
ORDER BY
    mb.runtime_minutes DESC
;"""
df_90120 = pd.read_sql(q, conn)
df_90120

In [ ]:
# z-test - 90 to 120 minutes ratings run

a = 0.025

# two-tailed z-test

x_bar = df_90120['AverageRating'].mean()
n = df_90120['AverageRating'].count()
std = df_allrt['AverageRating'].std()
mu = df_allrt['AverageRating'].mean()

z = (x_bar - mu)/(std/np.sqrt(n))
p = 1 - stats.norm.cdf(np.absolute(z))

# displaying results
print(f'Sample Mean: {x_bar} | Population Mean: {mu} | Sample Size: {n}')
print(f'p-value: {p} | z-score: {z} | Is p less than alpha? {p < a}')
if p < a:
    print("Reject the null")
elif p >= a:
    print("Fail to reject null")

In [ ]:
# normalizing distribution with for loop
samp_standardized_rating = [(x-mu)/std for x in df_90120['AverageRating']]
z_mean = np.mean(samp_standardized_rating)
print(f' Standardized Sample Mean: {z_mean}')

In [ ]:
# visualizing above distribution as histogram
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(samp_standardized_rating, bins=24, density=True, color='r')
ax.set_xlabel('Z Scores')
ax.set_title('90 to 120 minutes - Ratings')

# applying normalized curve over histogram
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

# marking standardized_mean
ax.vlines(x=z_mean, ymin=0, ymax=0.7, color='b', label='Mean: 0.0270')
ax.legend()

In [ ]:
# plotting tested runtime sample against population
fig, ax = plt.subplots(figsize=(10,8))

plt.hist(pop_standardized_rating, bins=45, density=True, color='b', alpha = 0.5)
sns.kdeplot(pop_standardized_rating, ax=ax, color='y')

plt.hist(samp_standardized_rating, bins=24, density=True, color='r', alpha = 0.5)
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

plt.vlines(x=z_mean, ymin=0, ymax=0.7, color='m', label='90 < x <= 120 Mean: 0.0270')
plt.vlines(x=pop_z_mean, ymin=0, ymax=0.7, label='Population Mean: 0.0')
                                              
plt.xlabel('Z Scores')
plt.title('90 to 120 minutes to All Films - Ratings')
plt.legend()

### Correlation

### Ho:
##### - There is no correlation between runtime and average rating for movies that are between 90 and 120 minutes. 

### Ha:
##### - There is a statistically significant correlation between runtime and average rating at the 95% level for movies that are between 90 and 120 minutes. 

In [ ]:
# r-critical value to test if found r is statistically signifcant
critical_r = 0.0118
# Source: https://www.calculators.tech/t-value-calculator
# (Unable to calculate by hand as I have been unable to find a direct formula to find this - must use either r-critical table or a r-crit calculator like the above source)
df = df_90120['Runtime'].count() - 2 # getting degrees of freedom to determine r-critical
df

In [ ]:
# calculating correlation

x = df_90120['Runtime']
y = df_90120['AverageRating']

r = x.corr(y) # result is the pearson correlation coefficient
r

print(f'Pearson Correlation: {r} | r-critical: {critical_r} | Is r greater? {abs(r) >= critical_r}')
if abs(r) >= critical_r:
    print("Reject the null")
elif abs(r) < critical_r:
    print("Fail to reject null")

In [ ]:
sns.lmplot(x="Runtime", y="AverageRating", data=df_90120, line_kws={'color': 'red'}, height=8, aspect=1.5);

### 120 Minutes and Up Testing

### Z-Test

### Ho
##### - There is no difference between the average rating of a movie that has a runtime of 120 minutes or greater and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the average rating of a movie that has a runtime of 120 minutes or greater and other movies in the population at the 95% level.

In [ ]:
# runtime sample 5 - long length films (120.0001 minutes and up films)
q = """
SELECT
    mb.movie_id
    ,mb.primary_title AS title
    ,mb.runtime_minutes AS Runtime
    ,mb.genres
    ,mr.averagerating AS AverageRating
    ,mr.numvotes AS Num_Votes
FROM
    movie_basics AS mb
    JOIN movie_ratings AS mr
        USING(movie_id)
WHERE
    CAST(Num_Votes AS int) > 30
    AND CAST(mb.runtime_minutes as float) BETWEEN 120.0001 AND 210
    AND mb.runtime_minutes IS NOT NULL
    AND mb.genres IS NOT NULL
ORDER BY
    mb.runtime_minutes DESC
;"""
df_120up = pd.read_sql(q, conn)
df_120up

In [ ]:
# z-test - 120 minutes and up ratings run

a = 0.025

# two-tailed z-test

x_bar = df_120up['AverageRating'].mean()
n = df_120up['AverageRating'].count()
std = df_allrt['AverageRating'].std()
mu = df_allrt['AverageRating'].mean()

z = (x_bar - mu)/(std/np.sqrt(n))
p = 1 - stats.norm.cdf(np.absolute(z))

# displaying results
print(f'Sample Mean: {x_bar} | Population Mean: {mu} | Sample Size: {n}')
print(f'p-value: {p} | z-score: {z} | Is p less than alpha? {p < a}')
if p < a:
    print("Reject the null")
elif p >= a:
    print("Fail to reject null")

In [ ]:
# normalizing distribution with for loop
samp_standardized_rating = [(x-mu)/std for x in df_120up['AverageRating']]
z_mean = np.mean(samp_standardized_rating)
print(f' Standardized Sample Mean: {z_mean}')

In [ ]:
# visualizing above distribution as histogram
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(samp_standardized_rating, bins=15, density=True, color='r')
ax.set_xlabel('Z Scores')
ax.set_title('120 Minutes and Up - Ratings')

# applying normalized curve over histogram
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

# marking standardized_mean
ax.vlines(x=z_mean, ymin=0, ymax=0.7, color='b', label='Mean: 0.2899')
ax.legend()

In [ ]:
# plotting tested runtime sample against population
fig, ax = plt.subplots(figsize=(10,8))

plt.hist(pop_standardized_rating, bins=45, density=True, color='b', alpha = 0.5)
sns.kdeplot(pop_standardized_rating, ax=ax, color='y')

plt.hist(samp_standardized_rating, bins=15, density=True, color='r', alpha = 0.5)
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

plt.vlines(x=z_mean, ymin=0, ymax=0.7, color='m', label='120 < x Mean: 0.2899')
plt.vlines(x=pop_z_mean, ymin=0, ymax=0.7, label='Population Mean: 0.0')
                                              
plt.xlabel('Z Scores')
plt.title('120 Minutes and Up to All Films - Ratings')
plt.legend()

### Correlation:

##### Ho:
##### - There is no correlation between runtime and average rating for movies that are 120 minutes or longer. 

##### Ha:
##### - There is a statistically significant correlation between runtime and average rating at the 95% level for movies that are 120 minutes or longer. 

In [ ]:
# r-critical value to test if found r is statistically signifcant
critical_r = 0.0235
# Source: https://www.calculators.tech/t-value-calculator
# (Unable to calculate by hand as I have been unable to find a direct formula to find this - must use either r-critical table or a r-crit calculator like the above source)
df = df_120up['Runtime'].count() - 2 # getting degrees of freedom to determine r-critical
df

In [ ]:
# calculating correlation

x = df_120up['Runtime']
y = df_120up['AverageRating']

r = x.corr(y) # result is the pearson correlation coefficient
r

print(f'Pearson Correlation: {r} | r-critical: {critical_r} | Is r greater? {abs(r) >= critical_r}')
if abs(r) >= critical_r:
    print("Reject the null")
elif abs(r) < critical_r:
    print("Fail to reject null")

In [ ]:
# plotting the regression showing the line of best fit with slope of r
sns.lmplot(x="Runtime", y="AverageRating", data=df_120up, line_kws={'color': 'red'}, height=8, aspect=1.5);

## Conclusions from Runtime Sample Testing for Average Ratings

Z-Tests:
- For films that are less than 40 minute runtimes, we can reject the null hypothesis with a statistically significant p-value of 0.0000001193, meaning that in 95% of cases we can expect our average ratings to be higher because of this set of runtimes.
    ##### - Reject the Null
- For films that have between 40 and 60 minute runtimes, we can reject the null hypothesis with a statistically significant p-value of 0.0, meaning that in 95% of cases we can expect our average ratings to be higher because of this set of runtimes.
    ##### - Reject the Null
- For films that have between 60 and 90 minute runtimes, we can reject the null hypothesis with a statistically significant p-value of 0.0, meaning that in 95% of cases we can expect our average ratings to be lower because of this set of runtimes.
    ##### - Reject the Null
- For films that have between 90 and 120 minute runtimes, we can reject the null hypothesis with a statistically significant p-value of 0.00008571, meaning that in 95% of cases we can expect our average ratings to be higher because of this set of runtimes.
    ##### - Reject the Null
- For films that have either 120 minute runtimes or longer, we can reject the null hypothesis with a statistically significant p-value of 0.0, meaning that in 95% of cases we can expect our average ratings to be higher because of this set of runtimes.
    ##### - Reject the Null

Correlation:
- In 95% of cases, movies that 90 minutes or longer have a weak positive correlation between their runtime and their ratings. 
- In 95% of cases, movies that last between 60 and 90 minutes have a weak negative correlation between their runtime and ratings.
- For movies that are less than 60 minutes long, we cannot conclude that there is a statistically significant correlation either.

For all films less than 3.5 hours long (where we cut off movie runtimes that client would be unlikely to want to pursue), there is a statistically significant weak positive correlation between runtime and average rating of a film in 95% of cases.

### Merging Data from SQL IMdb and Box Office Records (Elliot's Effort)

#### Domestic Revenue per Rating Table 
- Merging on title

In [ ]:
# merging dr_movie_db with the sql database data
domgross_im_db = dr_movie_db.merge(df_allrt,how='inner',on='title')
domgross_im_db

In [ ]:
# checking for unintentionally duplicated values
domgross_im_db.duplicated('title').value_counts() 

In [ ]:
# dropping found unintentionally duplicated values
domgross_im_db = domgross_im_db.drop_duplicates(subset='title')
domgross_im_db

### Hypothesis for Runtime and Domestic Gross Revenue

In [ ]:
# standardizing population for secondary visualizations to follow
mu = domgross_im_db['domestic_gross'].mean()
std = domgross_im_db['domestic_gross'].std()

pop_standardized_rating = [(x-mu)/std for x in domgross_im_db['domestic_gross']]
pop_z_mean = np.mean(pop_standardized_rating)
pop_z_mean

### Up to 40 Minutes Testing

### Z-Test

### Ho:
##### - There is no difference between the domestic revenue of a movie that has a runtime of 40 minutes or less and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the domestic revenue of a movie that has a runtime of between 40 minutes or less minutes and other movies in the population.

In [ ]:
# pulling sample based on runtime from combined data
domgross_40_db = domgross_im_db.loc[domgross_im_db['Runtime'] <= 40]
domgross_40_db

##### Unable to perform z-test for this as there are no rows in sample for this range.

### 40 to 60 Minutes Testing

### Z-Test

### Ho:
##### - There is no difference between the domestic revenue of a movie that has a runtime of between 40 minutes and 60 minutes and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the domestic revenue of a movie that has a runtime of between 40 minutes and 60 minutes and other movies in the population.

In [ ]:
# pulling sample based on runtime from combined data
domgross_4060_db = domgross_im_db.loc[(domgross_im_db['Runtime'] > 40) & (domgross_im_db['Runtime'] <= 60)]
domgross_4060_db

##### Unable to perform z-test for this as there are no rows in sample for this range.

### 60 to 90 Minutes Testing

### Z-Test

### Ho:
##### - There is no difference between the domestic revenue of a movie that has a runtime of between 60 minutes and 90 minutes and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the domestic revenue of a movie that has a runtime of between 60 minutes and 90 minutes and other movies in the population.

In [ ]:
# pulling sample based on runtime from combined data
domgross_6090_db = domgross_im_db.loc[(domgross_im_db['Runtime'] > 60) & (domgross_im_db['Runtime'] <= 90)]
domgross_6090_db

In [ ]:
# z-test - 60 minutes to 90 minutes domestic revenue run

a = 0.025

# two-tailed z-test
x_bar = domgross_6090_db['domestic_gross'].mean()
n = domgross_6090_db['domestic_gross'].count()
std = domgross_im_db['domestic_gross'].std()
mu = domgross_im_db['domestic_gross'].mean()

z = (x_bar - mu)/(std/np.sqrt(n))
p = 1 - stats.norm.cdf(np.absolute(z))

# displaying results
print(f'Sample Mean: {x_bar} | Population Mean: {mu} | Sample Size: {n}')
print(f'p-value: {p} | z-score: {z} | Is p less than alpha? {p < a}')
if p < a:
    print("Reject the null")
elif p >= a:
    print("Fail to reject null")

In [ ]:
# normalizing distribution with for loop
samp_standardized_rating = [(x-mu)/std for x in domgross_6090_db['domestic_gross']]
z_mean = np.mean(samp_standardized_rating)
print(f' Standardized Sample Mean: {z_mean}')

In [ ]:
# visualizing above distribution as histogram
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(samp_standardized_rating, bins=11, density=True, color='r')
ax.set_xlabel('Z Scores')
ax.set_title('60 to 90 Minutes - DM Revenue')

# applying normalized curve over histogram
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

# marking standardized_mean
ax.vlines(x=z_mean, ymin=0, ymax=0.7, color='b', label='Mean: -0.2567')
ax.legend()

In [ ]:
# plotting tested runtime sample against population
fig, ax = plt.subplots(figsize=(10,8))

plt.hist(pop_standardized_rating, bins=16, density=True, color='b', alpha = 0.5)
sns.kdeplot(pop_standardized_rating, ax=ax, color='y')

plt.hist(samp_standardized_rating, bins=11, density=True, color='r', alpha = 0.5)
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

plt.vlines(x=z_mean, ymin=0, ymax=0.7, color='m', label='60 < x <= 90 Mean: -0.2567')
plt.vlines(x=pop_z_mean, ymin=0, ymax=0.7, label='Population Mean: 0.0')
                                              
plt.xlabel('Z Scores')
plt.title('60 to 90 Minutes to All Films - DM Revenue')
plt.legend()

### 90 to 120 Minutes Testing

### Z-Test

### Ho:
##### - There is no difference between the domestic revenue of a movie that has a runtime of between 90 minutes and 120 minutes and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the domestic revenue of a movie that has a runtime of between 90 minutes and 120 minutes and other movies in the population.

In [ ]:
# pulling sample based on runtime from combined data
domgross_90120_db = domgross_im_db.loc[(domgross_im_db['Runtime'] > 90) & (domgross_im_db['Runtime'] <= 120)]
domgross_90120_db

In [ ]:
# z-test - 90 minutes to 120 minutes domestic revenue run

a = 0.025

# two-tailed z-test
x_bar = domgross_90120_db['domestic_gross'].mean()
n = domgross_90120_db['domestic_gross'].count()
std = domgross_im_db['domestic_gross'].std()
mu = domgross_im_db['domestic_gross'].mean()

z = (x_bar - mu)/(std/np.sqrt(n))
p = 1 - stats.norm.cdf(np.absolute(z))

# displaying results
print(f'Sample Mean: {x_bar} | Population Mean: {mu} | Sample Size: {n}')
print(f'p-value: {p} | z-score: {z} | Is p less than alpha? {p < a}')
if p < a:
    print("Reject the null")
elif p >= a:
    print("Fail to reject null")

In [ ]:
# normalizing distribution with for loop
samp_standardized_rating = [(x-mu)/std for x in domgross_90120_db['domestic_gross']]
z_mean = np.mean(samp_standardized_rating)
print(f' Standardized Sample Mean: {z_mean}')

In [ ]:
# visualizing above distribution as histogram
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(samp_standardized_rating, bins=22, density=True, color='r')
ax.set_xlabel('Z Scores')
ax.set_title('90 to 120 Minutes - DM Revenue')

# applying normalized curve over histogram
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

# marking standardized_mean
ax.vlines(x=z_mean, ymin=0, ymax=0.7, color='b', label='Mean: -0.04584')
ax.legend()

In [ ]:
# plotting tested runtime sample against population
fig, ax = plt.subplots(figsize=(10,8))

plt.hist(pop_standardized_rating, bins=45, density=True, color='b', alpha = 0.5)
sns.kdeplot(pop_standardized_rating, ax=ax, color='y')

plt.hist(samp_standardized_rating, bins=15, density=True, color='r', alpha = 0.5)
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

plt.vlines(x=z_mean, ymin=0, ymax=0.7, color='m', label='90 < x <= 120 Mean: -0.04584')
plt.vlines(x=pop_z_mean, ymin=0, ymax=0.7, label='Population Mean: 0.0')
                                              
plt.xlabel('Z Scores')
plt.title('90 to 120 Minutes to All Films - DM Revenue')
plt.legend()

### 120 Minutes and Up Testing

### Z-Test

### Ho:
##### - There is no difference between the domestic revenue of a movie that has a runtime of 120 minutes or greater and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the domestic revenue of a movie that has a runtime of 120 minutes or greater and other movies in the population at the 95% level.

In [ ]:
# pulling sample based on runtime from combined data
domgross_120up_db = domgross_im_db.loc[domgross_im_db['Runtime'] > 120]
domgross_120up_db

In [ ]:
# z-test - 120 minutes and up domestic revenue run

a = 0.025

# two-tailed z-test
x_bar = domgross_120up_db['domestic_gross'].mean()
n = domgross_120up_db['domestic_gross'].count()
std = domgross_im_db['domestic_gross'].std()
mu = domgross_im_db['domestic_gross'].mean()

z = (x_bar - mu)/(std/np.sqrt(n))
p = 1 - stats.norm.cdf(np.absolute(z))

# displaying results
print(f'Sample Mean: {x_bar} | Population Mean: {mu} | Sample Size: {n}')
print(f'p-value: {p} | z-score: {z} | Is p less than alpha? {p < a}')
if p < a:
    print("Reject the null")
elif p >= a:
    print("Fail to reject null")

In [ ]:
# normalizing distribution with for loop
samp_standardized_rating = [(x-mu)/std for x in domgross_120up_db['domestic_gross']]
z_mean = np.mean(samp_standardized_rating)
print(f' Standardized Sample Mean: {z_mean}')

In [ ]:
# visualizing above distribution as histogram
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(samp_standardized_rating, bins=11, density=True, color='r')
ax.set_xlabel('Z Scores')
ax.set_title('120 Minutes and Up - DM Revenue')

# applying normalized curve over histogram
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

# marking standardized_mean
ax.vlines(x=z_mean, ymin=0, ymax=0.7, color='b', label='Mean: 0.3130')
ax.legend()

In [ ]:
# plotting tested runtime sample against population
fig, ax = plt.subplots(figsize=(10,8))

plt.hist(pop_standardized_rating, bins=45, density=True, color='b', alpha = 0.5)
sns.kdeplot(pop_standardized_rating, ax=ax, color='y')

plt.hist(samp_standardized_rating, bins=15, density=True, color='r', alpha = 0.5)
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

plt.vlines(x=z_mean, ymin=0, ymax=0.7, color='m', label='120 < x Mean: 0.3130')
plt.vlines(x=pop_z_mean, ymin=0, ymax=0.7, label='Population Mean: 0.0')
                                              
plt.xlabel('Z Scores')
plt.title('120 Minutes and Up to All Films - DM Revenue')
plt.legend()

### Correlation

### Ho:
##### - There is no correlation between runtime and the domestic gross revenue for a given movie at the 95% level.  

### Ha:
##### - There is a statistically significant correlation between runtime and the domestic gross revenue at the 95% level for a given movie.

In [ ]:
# r-critical value to test if found r is statistically signifcant
critical_r = 0.0510
# Source: https://www.calculators.tech/t-value-calculator
# (Unable to calculate by hand as I have been unable to find a direct formula to find this - must use either r-critical table or a r-crit calculator like the above source)
df = domgross_im_db['Runtime'].count() - 2 # getting degrees of freedom to determine r-critical
df

In [ ]:
# calculating correlation

x = domgross_im_db['Runtime']
y = domgross_im_db['domestic_gross']

r = x.corr(y) # result is the pearson correlation coefficient
r

print(f'Pearson Correlation: {r} | r-critical: {critical_r} | Is r greater? {abs(r) >= critical_r}')
if abs(r) >= critical_r:
    print("Reject the null")
elif abs(r) < critical_r:
    print("Fail to reject null")

In [ ]:
sns.lmplot(x="Runtime", y="domestic_gross", data=domgross_im_db, line_kws={'color': 'red'}, height=8, aspect=1.5);

#### Worldwide Revenue per Rating Table 
- Merging on title

In [ ]:
# merging dr_movie_db with the sql database data
wwgross_im_db = wwr_movie_db.merge(df_allrt,how='inner',on='title')
wwgross_im_db

In [ ]:
# checking for unintentionally duplicated values
wwgross_im_db.duplicated('title').value_counts() 

In [ ]:
# dropping found unintentionally duplicated values
wwgross_im_db = wwgross_im_db.drop_duplicates(subset='title')
wwgross_im_db

### Hypothesis Testing for Runtime and Worldwide Gross Revenue

In [ ]:
# standardizing population for secondary visualizations to follow
mu = wwgross_im_db['worldwide_gross'].mean()
std = wwgross_im_db['worldwide_gross'].std()

pop_standardized_rating = [(x-mu)/std for x in wwgross_im_db['worldwide_gross']]
pop_z_mean = np.mean(pop_standardized_rating)
pop_z_mean

### Less than 40 Minutes Testing

### Z-Test

### Ho:
##### - There is no difference between the worldwide revenue of a movie that has a runtime of less than 40 minutes and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the worldwide revenue of a movie that has a runtime of less than 40 minutes and the population. 

In [ ]:
wwgross_40_db = wwgross_im_db.loc[wwgross_im_db['Runtime'] <= 40]
wwgross_40_db

##### Unable to perform z-test for this as there are no rows in sample for this range.

### 40 to 60 Minutes Testing

### Z-Test

### Ho:
##### - There is no difference between the worldwide revenue of a movie that has a runtime of between 40 minutes and 60 minutes and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the worldwide revenue of a movie that has a runtime of between 40 minutes and 60 minutes and other movies in the population.

In [ ]:
wwgross_4060_db = wwgross_im_db.loc[(wwgross_im_db['Runtime'] > 40) & (wwgross_im_db['Runtime'] <= 60)]
wwgross_4060_db

##### Unable to perform z-test for this as there are no rows in sample for this range.

### 60 to 90 Minutes Testing

### Z-Test

### Ho:
##### - There is no difference between the worldwide revenue of a movie that has a runtime of between 60 minutes and 90 minutes and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the worldwide revenue of a movie that has a runtime of between 60 minutes and 90 minutes and other movies in the population.

In [ ]:
wwgross_6090_db = wwgross_im_db.loc[(wwgross_im_db['Runtime'] > 60) & (wwgross_im_db['Runtime'] <= 90)]
wwgross_6090_db

In [ ]:
# z-test - 60 minutes to 90 minutes worldwide revenue run

a = 0.025

# two-tailed z-test
x_bar = wwgross_6090_db['worldwide_gross'].mean()
n = wwgross_6090_db['worldwide_gross'].count()
std = wwgross_im_db['worldwide_gross'].std()
mu = wwgross_im_db['worldwide_gross'].mean()

z = (x_bar - mu)/(std/np.sqrt(n))
p = 1 - stats.norm.cdf(np.absolute(z))

# displaying results
print(f'Sample Mean: {x_bar} | Population Mean: {mu} | Sample Size: {n}')
print(f'p-value: {p} | z-score: {z} | Is p less than alpha? {p < a}')
if p < a:
    print("Reject the null")
elif p >= a:
    print("Fail to reject null")

In [ ]:
# normalizing distribution with for loop
samp_standardized_rating = [(x-mu)/std for x in wwgross_6090_db['worldwide_gross']]
z_mean = np.mean(samp_standardized_rating)
print(f' Standardized Sample Mean: {z_mean}')

In [ ]:
# visualizing above distribution as histogram
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(samp_standardized_rating, bins=11, density=True, color='r')
ax.set_xlabel('Z Scores')
ax.set_title('60 to 90 Minutes - WW Revenue')

# applying normalized curve over histogram
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

# marking standardized_mean
ax.vlines(x=z_mean, ymin=0, ymax=0.7, color='b', label='Mean: -0.3105')
ax.legend()

In [ ]:
# plotting tested runtime sample against population
fig, ax = plt.subplots(figsize=(10,8))

plt.hist(pop_standardized_rating, bins=16, density=True, color='b', alpha = 0.5)
sns.kdeplot(pop_standardized_rating, ax=ax, color='y')

plt.hist(samp_standardized_rating, bins=11, density=True, color='r', alpha = 0.5)
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

plt.vlines(x=z_mean, ymin=0, ymax=0.7, color='m', label='60 < x <= 90 Mean: -0.3105')
plt.vlines(x=pop_z_mean, ymin=0, ymax=0.7, label='Population Mean: 0.0')
                                              
plt.xlabel('Z Scores')
plt.title('60 to 90 Minutes to All Films - WW Revenue')
plt.legend()

### 90 to 120 Minutes Testing

### Z-Test

### Ho:
##### - There is no difference between the worldwide revenue of a movie that has a runtime of between 90 minutes and 120 minutes and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the worldwide revenue of a movie that has a runtime of between 90 minutes and 120 minutes and other movies in the population.

In [ ]:
wwgross_90120_db = wwgross_im_db.loc[(wwgross_im_db['Runtime'] > 90) & (wwgross_im_db['Runtime'] <= 120)]
wwgross_90120_db

In [ ]:
# z-test - 90 minutes to 120 minutes worldwide revenue run

a = 0.025

# two-tailed z-test
x_bar = wwgross_90120_db['worldwide_gross'].mean()
n = wwgross_90120_db['worldwide_gross'].count()
std = wwgross_im_db['worldwide_gross'].std()
mu = wwgross_im_db['worldwide_gross'].mean()

z = (x_bar - mu)/(std/np.sqrt(n))
p = 1 - stats.norm.cdf(np.absolute(z))

# displaying results
print(f'Sample Mean: {x_bar} | Population Mean: {mu} | Sample Size: {n}')
print(f'p-value: {p} | z-score: {z} | Is p less than alpha? {p < a}')
if p < a:
    print("Reject the null")
elif p >= a:
    print("Fail to reject null")

In [ ]:
# normalizing distribution with for loop
samp_standardized_rating = [(x-mu)/std for x in wwgross_90120_db['worldwide_gross']]
z_mean = np.mean(samp_standardized_rating)
print(f' Standardized Sample Mean: {z_mean}')

In [ ]:
# visualizing above distribution as histogram
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(samp_standardized_rating, bins=15, density=True, color='r')
ax.set_xlabel('Z Scores')
ax.set_title('90 to 120 Minutes - WW Revenue')

# applying normalized curve over histogram
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

# marking standardized_mean
ax.vlines(x=z_mean, ymin=0, ymax=0.7, color='b', label='Mean: -0.0203')
ax.legend()

In [ ]:
# plotting tested runtime sample against population
fig, ax = plt.subplots(figsize=(10,8))

plt.hist(pop_standardized_rating, bins=45, density=True, color='b', alpha = 0.5)
sns.kdeplot(pop_standardized_rating, ax=ax, color='y')

plt.hist(samp_standardized_rating, bins=15, density=True, color='r', alpha = 0.5)
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

plt.vlines(x=z_mean, ymin=0, ymax=0.7, color='m', label='90 < x <= 120 Mean: -0.0203')
plt.vlines(x=pop_z_mean, ymin=0, ymax=0.7, label='Population Mean: 0.0')
                                              
plt.xlabel('Z Scores')
plt.title('90 to 120 Minutes to All Films - WW Revenue')
plt.legend()

### 120 and up Minutes Testing

### Z-Test

### Ho:
##### - There is no difference between the worldwide revenue of a movie that has a runtime of 120 minutes or greater and other movies in the population.

### Ha:
##### - There is a statistically significant difference between the worldwide revenue of a movie that has a runtime of 120 minutes or greater and other movies in the population at the 95% level.

In [ ]:
wwgross_120up_db = wwgross_im_db.loc[wwgross_im_db['Runtime'] > 120]
wwgross_120up_db

In [ ]:
# z-test - 120 minutes and up worldwide revenue run

a = 0.025

# two-tailed z-test
x_bar = wwgross_120up_db['worldwide_gross'].mean()
n = wwgross_120up_db['worldwide_gross'].count()
std = wwgross_im_db['worldwide_gross'].std()
mu = wwgross_im_db['worldwide_gross'].mean()

z = (x_bar - mu)/(std/np.sqrt(n))
p = 1 - stats.norm.cdf(np.absolute(z))

# displaying results
print(f'Sample Mean: {x_bar} | Population Mean: {mu} | Sample Size: {n}')
print(f'p-value: {p} | z-score: {z} | Is p less than alpha? {p < a}')
if p < a:
    print("Reject the null")
elif p >= a:
    print("Fail to reject null")

In [ ]:
# normalizing distribution with for loop
samp_standardized_rating = [(x-mu)/std for x in wwgross_120up_db['worldwide_gross']]
z_mean = np.mean(samp_standardized_rating)
print(f' Standardized Sample Mean: {z_mean}')

In [ ]:
# visualizing above distribution as histogram
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(samp_standardized_rating, bins=15, density=True, color='r')
ax.set_xlabel('Z Scores')
ax.set_title('120 Minutes and Up - WW Revenue')

# applying normalized curve over histogram
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

# marking standardized_mean
ax.vlines(x=z_mean, ymin=0, ymax=0.7, color='b', label='Mean: 0.2631')
ax.legend()

In [ ]:
# plotting tested runtime sample against population
fig, ax = plt.subplots(figsize=(10,8))

plt.hist(pop_standardized_rating, bins=45, density=True, color='b', alpha = 0.5)
sns.kdeplot(pop_standardized_rating, ax=ax, color='y')

plt.hist(samp_standardized_rating, bins=15, density=True, color='r', alpha = 0.5)
sns.kdeplot(samp_standardized_rating, ax=ax, color='k')

plt.vlines(x=z_mean, ymin=0, ymax=0.7, color='m', label='120 < x Mean: 0.2631')
plt.vlines(x=pop_z_mean, ymin=0, ymax=0.7, label='Population Mean: 0.0')
                                              
plt.xlabel('Z Scores')
plt.title('120 Minutes and Up to All Films - WW Revenue')
plt.legend()

### Domestic Gross Z-Tests:
- For films that are less than 40 minute runtimes, we cannot perform a hypothesis test because our filter returns no values in this runtime range.
    ##### - Inconclusive
- For films that have between 40 and 60 minute runtimes, we cannot perform a hypothesis test because our filter returns no values in this runtime range.
    ##### - Inconclusive
- For films that have between 60 and 90 minute runtimes, we can reject the null hypothesis with a statistically significant p-value of 0.0026, meaning that in 95% of cases we can expect our revenue to be lower (due to negative z-score) if a film falls within this set of runtimes.
    ##### - Reject the Null
- For films that have between 90 and 120 minute runtimes, we fail to reject the null hypothesis for a p-value of 0.1094 because it is greater than our alpha 0.025.
    ##### - Fail to reject the Null
- For films that have either 120 minute runtimes or longer, we can reject the null hypothesis with a statistically significant p-value of 0.000004, meaning that in 95% of cases we can expect our revenue to be higher (due to positive z-score) because of the movie falling this range of runtimes.
    ##### - Reject the Null
    
### Worldwide Gross Z-Tests:
- For films that are less than 40 minute runtimes, we cannot perform a hypothesis test because our filter returns no values in this runtime range.
    ##### - Inconclusive
- For films that have between 40 and 60 minute runtimes, we can reject the null hypothesis with a statistically significant p-value of 0.0, meaning that in 95% of cases we can expect our average ratings to be higher because of this set of runtimes.
    ##### - Inconclusive
- For films that have between 60 and 90 minute runtimes, we can reject the null hypothesis with a statistically significant p-value of 0.0004, meaning that in 95% of cases we can expect our revenue to be lower (due to negative z-score) if a film falls within this set of runtimes.
    ##### - Reject the Null
- For films that have between 90 and 120 minute runtimes, we fail to reject the null hypothesis for a p-value of 0.2937 because it is greater than our alpha 0.025.
    ##### - Fail to reject the Null
- For films that have either 120 minute runtimes or longer, we can reject the null hypothesis with a statistically significant p-value of 0.0001, meaning that in 95% of cases we can expect our revenue to be higher (due to positive z-score) because of the movie falling this range of runtimes.
    ##### - Reject the Null

### Correlation for Worldwide Gross Revenue

### Ho:
##### - There is no correlation between runtime and the worldwide gross revenue for a given movie at the 95% level.  

### Ha:
##### - There is a statistically significant correlation between runtime and the worldwide gross revenue at the 95% level for a given movie.

In [ ]:
# r-critical value to test if found r is statistically signifcant
critical_r = 0.0515
# Source: https://www.calculators.tech/t-value-calculator
# (Unable to calculate by hand as I have been unable to find a direct formula to find this - must use either r-critical table or a r-crit calculator like the above source)
df = wwgross_im_db['Runtime'].count() - 2 # getting degrees of freedom to determine r-critical
df

In [ ]:
# calculating correlation

x = wwgross_im_db['Runtime']
y = wwgross_im_db['worldwide_gross']

r = x.corr(y) # result is the pearson correlation coefficient
r

print(f'Pearson Correlation: {r} | r-critical: {critical_r} | Is r greater? {abs(r) >= critical_r}')
if abs(r) >= critical_r:
    print("Reject the null")
elif abs(r) < critical_r:
    print("Fail to reject null")

In [ ]:
sns.lmplot(x="Runtime", y="worldwide_gross", data=wwgross_im_db, line_kws={'color': 'red'}, height=8, aspect=1.5);

## Frida

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## Enrique

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## John

In [ ]:
# Your code here - remember to use markdown cells for comments as well!